In [1]:
import numpy as np
import math
import copy

In [152]:
optimized_flag = False

In [131]:
def coord_to_array(a, i, j):
    global optimized_flag
    
    if optimized_flag == False:
        return a[i][j]
    else:
        if j > i:
            aux = i
            i = j
            j = aux
        return a[i*(i+1)//2 + j]

In [162]:
def set_to_array(a, i, j, val):
    global optimized_flag

    if optimized_flag == False:
        a[i][j] = val
        a[j][i] = val
    else:
        if j > i:
            aux = i
            i = j
            j = aux
        a[i*(i+1)//2 + j] = val

In [133]:
def coord_to_array_l(l, i, j):
    global optimized_flag
    if j > i:
        return 0
    if optimized_flag == False:
        return l[i][j]
    return l[i*(i+1)//2 + j]

In [134]:
def read_input(a, n):
    global optimized_flag
    file_read = open("input.txt", "r")
    n = int(file_read.readline())
    
    if optimized_flag == True:
        for i in range(0, n*(n+1)//2):
            value = float(file_read.readline())
            a = np.append(a, value)
    else:
        a = np.zeros((n,n))
        for i in range(0, n):
            for j in range(0, i+1):
                value = float(file_read.readline())
                set_to_array(a, i, j, value)
                set_to_array(a, j, i, value)
            
    return (a,n)

In [109]:
def initialize_u(u, n, length):
    for i in range(0, n):
        u[i][i] = 1
    return u

In [110]:
def compute_pq(a, n):
    maxim = -1
    for i in range(0, n):
        for j in range(0, i):
            if abs(coord_to_array(a, i,j)) > maxim:
                maxim = abs(coord_to_array(a,i,j))
                p = i
                q = j
    return (p,q)

In [111]:
def compute_tcs(a, n, p, q):
    alfa = (coord_to_array(a,p,p) - coord_to_array(a,q,q))/(2*coord_to_array(a,p,q))
    if alfa >= 0:
        t = -alfa + math.sqrt(alfa*alfa+1)
    else:
        t = -alfa - math.sqrt(alfa*alfa+1)
    
    c = 1 / math.sqrt(1 + t*t)
    s = t / math.sqrt(1 + t*t)
    
    return(t,c,s)

In [112]:
def next_matrix_a(a, n, p, q, t, c, s):
    a_aux = copy.deepcopy(a)
    
    for i in range(0, n):
        if i != p and i != q:
#             if p > i:
            set_to_array(a, p, i, c*coord_to_array(a_aux,p,i) + s*coord_to_array(a_aux,q,i))
    
    for i in range(0, n):
        if i != p and i != q:
            set_to_array(a, q, i, -s*coord_to_array(a_aux,i,p) + c*coord_to_array(a_aux,q,i))
#             if q > i:
#                 coord_to_array(a,q,i) = -s*coord_to_array(a_aux,i,p) + c*coord_to_array(a_aux,q,i)
#             if i > q:
#                 a[coord_to_array(i,q)] = -s*a[coord_to_array(i,p)] + c*a[coord_to_array(q,i)]

#     for i in range(0, n):
#         if i != p and i != q:
#             if i > p:
#                 a[coord_to_array(i,p)] = a[coord_to_array(p,i)]
            
    set_to_array(a, p, p, coord_to_array(a_aux,p,p) + t*coord_to_array(a_aux,p,q))
    set_to_array(a, q, q, coord_to_array(a_aux,q,q) - t*coord_to_array(a_aux,p,q))
    set_to_array(a, p, q, 0)
    set_to_array(a, q, p, 0)

    return a

In [113]:
def next_matrix_u(u, n, p, q, t, c, s):
    for i in range(0, n):
        uip_vechi = copy.deepcopy(u[i][p])
        u[i][p] = c * u[i][p] + s * u[i][q]
        u[i][q] = -s * uip_vechi + c * u[i][q]
    return u

In [114]:
def get_error(a_init, n, vp, u):

    error = 0
    au = np.zeros((n,n))
    uvp = np.zeros((n,n))
    
    for i in range(0,n):
        for j in range(0,n):
            value = 0
            for t in range(0,n):
                au[j][i] += coord_to_array(a_init,j,t) * u[t][i]

    for i in range(0, n):
        for j in range(0, n):
            uvp[i][j] = u[i][j] * vp[j]
            
    for i in range(0,n):
        for j in range(0,n):
            error += abs(au[i][j] - uvp[i][j])
    error = math.sqrt(error)
    print("ERROR: ", error)

In [166]:
def get_valori_proprii():
    global optimized_flag
    n = -1
    k = 0
    kmax = 1000
    epsilon = 10 ** (-8)
    
    a = np.array([])
    (a,n) = read_input(a,n)
    u = np.zeros((n,n))
    a_init = copy.deepcopy(a)
    length = len(a)
    u = initialize_u(u, n, length)
    (p,q) = compute_pq(a,n)
    (t,c,s) = compute_tcs(a,n,p,q)
    
    while abs(coord_to_array(a,p,q)) > epsilon and k<kmax:
        a = next_matrix_a(a, n, p, q, t, c, s)
        u = next_matrix_u(u, n, p, q, t, c, s)
        (p,q) = compute_pq(a,n)
        (t,c,s) = compute_tcs(a,n,p,q)
        
        k = k+1

#         valori_proprii = np.array([])
#         for i in range(0, n):
#             valori_proprii = np.append(valori_proprii, a[coord_to_array(i,i)])
#         print(valori_proprii)

    valori_proprii = np.array([])
    for i in range(0, n):
        valori_proprii = np.append(valori_proprii, coord_to_array(a,i,i))
    
    get_error(a_init, n, valori_proprii, u)
    
    return (a, a_init, u, valori_proprii)

(a, a_init, u, vp) = get_valori_proprii()
print(vp)



ERROR:  0.00027283259983755336
[ 2.69838866  0.52621324 32.0878981 ]


In [170]:
def cerinta2():
    k = 0
    n = -1
    kmax = 1000
    epsilon = 10 ** (-12)
    a = np.array([])
    (a,n) = read_input(a,n)
    
    l = compute_l(a, n)
    a_next = product_l(l,n)
    
    while(compare_a(a, a_next, n) > epsilon and k < kmax):
        a = a_next
        l = compute_l(a, n)
        a_next = product_l(l,n)
        k += 1
        
        
    vp = np.array([])
    for i in range(0, n):
        vp = np.append(vp, coord_to_array(a_next, i, i))
        
    return vp
    
cerinta2()
    

array([32.0878981 ,  2.69838866,  0.52621324])

In [142]:
def compute_l(a, n):
    global optimized_flag
    
    if optimized_flag == True:
        l = np.array([])
        for i in range(0, int(n*(n+1)/2)):
            l = np.append(l, 0)
    else:
        l = np.zeros((n,n))
        
    for i in range(0, n):
        compute_l_column_p(i, l, a, n)
        
    return l

In [143]:
def compute_l_column_p(column, l, a, n):    
    suma = 0
    for i in range(0, column):
        suma += coord_to_array_l(l,column,i) * coord_to_array_l(l,column,i)
    if coord_to_array(a,column,column) - suma <= 0:
        print('MATRICEA NU ESTE POZITIV DEFINITA!')

    val = math.sqrt(coord_to_array(a,column,column) - suma)
    set_to_array(l, column, column, val)
    
    
    for i in range(column+1, n):
        suma = 0
        for j in range(0, column):
            suma += coord_to_array_l(l,i,j) * coord_to_array_l(l,column, j)
            
        val = coord_to_array(a,i,column) - suma
        val = val / coord_to_array_l(l,column,column)
        set_to_array(l, i, column, val)


In [144]:
def product_l(l,n):
    global optimized_flag
    
    if optimized_flag == True:
        new_a = np.array([])
        for i in range(0, int(n*(n+1)/2)):
            new_a = np.append(new_a, 0)
    else:
        new_a = np.zeros((n,n))
        
    for i in range(0,n):
        for j in range(0,i+1):
            suma = 0
            for t in range(0,n):
                suma += coord_to_array_l(l,t,j) * coord_to_array_l(l,t,i)
            set_to_array(new_a, j, i, copy.deepcopy(suma))
                
    return new_a

In [145]:
def compare_a(a1, a2, n):
    error = 0
    for i in range(0,n):
        for j in range(0,n):
            error += abs(coord_to_array(a1,i,j) - coord_to_array(a2,i,j))
    return math.sqrt(error)

In [173]:
def get_valori_singulare():
    epsilon = 10 ** (-12)
    n = -1
    a = np.array([])
    (a,n) = read_input(a,n)
    
    (u, s, v) = np.linalg.svd(a)
    return s
    
    
get_valori_singulare()

array([32.0878981 ,  2.69838866,  0.52621324])

In [174]:
def get_rang_numpy():
    n = -1
    a = np.array([])
    (a,n) = read_input(a,n)
    return np.linalg.matrix_rank(a)

get_rang_numpy()

3

In [175]:
def get_rang_formula():
    n = -1
    a = np.array([])
    (a,n) = read_input(a,n)
    (u, s, v) = np.linalg.svd(a)
    count = 0
    for i in range(0, n):
        if s[i] > 0:
            count += 1
    return count

get_rang_formula()

3

In [177]:
def get_numar_conditionare_numpy():
    n = -1
    a = np.array([])
    (a,n) = read_input(a,n)
    return np.linalg.cond(a)

get_numar_conditionare_numpy()

60.97888744877879

In [179]:
def get_numar_conditionare_formula():
    n = -1
    a = np.array([])
    (a,n) = read_input(a,n)
    (u, s, v) = np.linalg.svd(a)
    
    maxim = -1
    minim = 999999999
    for i in range(0,n):
        if s[i] > maxim:
            maxim = s[i]
        if s[i] < minim and s[i] > 0:
            minim = s[i]
            
    return maxim/minim

get_numar_conditionare_formula()

60.97888744877878

In [202]:
def moore_penrose():
    epsilon = 10 ** (-8)
    n = -1
    a = np.array([])
    (a,n) = read_input(a,n)
    (u, s, v) = np.linalg.svd(a)
    for i in range(0, n):
        if abs(s[i]) > epsilon:
            s[i] = 1 / s[i]
            
    sinv = np.zeros((n,n))
    for i in range(0, n):
        sinv[i][i] = s[i]
    
    return v.T.dot(sinv).dot(u.T)

moore_penrose()

array([[ 1.06447188, -0.72427984,  0.25925926],
       [-0.72427984,  0.98765432, -0.44444444],
       [ 0.25925926, -0.44444444,  0.25      ]])

In [183]:
def moore_penrose2():
    n = -1
    a = np.array([])
    (a,n) = read_input(a,n)
    
    result = copy.deepcopy(np.transpose(a))
    result = np.dot(result, a)
    result = np.linalg.inv(result)
    result = np.dot(result, np.transpose(a))
    return result

moore_penrose2()

array([[ 1.06447188, -0.72427984,  0.25925926],
       [-0.72427984,  0.98765432, -0.44444444],
       [ 0.25925926, -0.44444444,  0.25      ]])

In [203]:
def compare_inverses():
    n = -1
    a = np.array([])
    (a,n) = read_input(a,n)
    matrix1 = moore_penrose()
    matrix2 = moore_penrose2()
    error = 0
    
    for i in range(0, n):
        for j in range(0, n):
            error += abs(matrix1[i][j] - matrix2[i][j])
    
    return math.sqrt(error)

compare_inverses()

4.622963684048823e-07